# SAT Machine Learning Hackathon Team $\Sigma \Omega$

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import brier_score_loss
from sklearn.model_selection import train_test_split

# The following option ensures that Pandas treats string as string and not "objects"
# This is something that will be default behaviour in Pandas 3.0, but you can already
# switch it on.
pd.options.future.infer_string = True

# Import the dataset, casting some of the categorical features that are stored as numbers
# as strings, rather than allowing Pandas to assuming that they are meant to be
# integers or floats.
df = (
    pd.read_csv(
        'data/train.csv', 
        dtype={
            'attendance_category': 'str', 
            'treatment_function_code': 'str', 
            'palliative_care_description': 'str',
            }
        )
)
 

In [ ]:
# Have a look at our data. Notice that it shows the first dozen columns and 
# then the last dozen...
df.head()

In [ ]:
# Using .iloc means that we can take vertical slices of our DataFrame, so that
# we can look at all of our columns over a few code cells.
# The syntax is df.iloc[rows,columns]. The ":"" for rows means "everything" and the
# ":10" means "up to the 10th column".

df.iloc[:,:10].head()

In [ ]:
# The 11th to the 20th columns
df.iloc[:,11:20].head()

In [ ]:
# The 21st to the 30th columns
df.iloc[:,21:30].head()

In [ ]:
# Column 31 to the last column
df.iloc[:,31:].head()

In [ ]:
# See how many rows and columns there are
df.shape

In [ ]:
# This will give us the data types and how many values are not null
df.info()

# EDA

In [ ]:
# This is a nifty way of getting the percentage of values that are
# missing from each column
df.isnull().mean() * 100

In [ ]:
# This is a simple visualisation of how many values are missing from each column.
fig, ax = plt.subplots(figsize=(12,4))
 
(1 - df.isnull().mean()).abs().plot.bar(ax=ax)
 
plt.show()

In [ ]:
# Access the "columns" attribute of the dataframe to return the column names,
# which can be copied into your code when exploring the data.
# Note that this isn't strictly a Python "list". You would need to cast it as
# such: list(df.columns)
df.columns

### Creating charts to explore data fields.

Here we are going to create a chart of how many values there are for each category in "acuity_desc". Firstly, we create a DataFrame and then we can put it into a simple `seaborn` chart, using `matplotlib` to tweak the parameters.

In [ ]:
acuity = pd.DataFrame(df.acuity_desc.value_counts())

acuity.reset_index(inplace=True)

In [ ]:
acuity

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
sns.barplot(acuity, x='acuity_desc', y='count', ax=ax)
plt.xticks(rotation=90)
plt.show()

Creating a facet grid to look at the distribution of multiple numeric values.

_We didn't all get to this on the Hackathon day, but here are is a way to do it since it's a handy way to appraise multiple fields in one go._

In [ ]:
# Take some numeric fields that have a bit of a range to them.

num_fields_facet = ['departure_time_since_arrival','lsoa_site_of_treatment_distance','age_at_arrival']

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols= len(num_fields_facet), figsize=(12,4))

for i, col in enumerate(num_fields_facet):
    sns.histplot(data=df, x=col, ax=axes[i])
    axes[i].set_title(col)

plt.suptitle('Numerical Feature Distributions')
plt.tight_layout()
plt.show()

A version that looks at the value counts of categorical fields.

In [ ]:
# Below is how to select all of the categorical fields in the dataset,
# but we will select a few on this occasion since our dataset is large
# and has a large number of categorical features.

# get the categorical (string) fields in the dataset.
# cat_fields_facet = list(df.select_dtypes(str).columns)

cat_fields_facet = [
    'ethnic_category',
    'patient_status',
    'acuity_desc',
    'care_home_status',
    'destination_desc',
    'arrival_mode_desc'
]

Plotting the grid of charts. Note that it gets quite big due to the length of the category names. You may wish to do some processing to reduce the length of the categories, possibly by encoding them with single letters or numbers.

In [ ]:
n = len(cat_fields_facet)               # number of features being examined
ncols = 3                               # number of columns in the grid
nrows = np.ceil(n/ncols).astype(int)    # number of rows required to cover all the features, accounting for the number of columns.

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20, 8*nrows))   # set your plot area, figsize=(width,height)

axes = axes.flatten()   # this is needed because the output of the rows/columns calculation is a numpy array, which
                        # can't be used in an Axes object.

for ax, col in zip(axes, cat_fields_facet):
    counts = df[col].value_counts().reset_index()
    counts.columns = [col, "count"]

    sns.barplot(data=counts, x=col, y="count", ax=ax)
    ax.set_title(f"Value Counts for {col}")
    ax.set_xlabel("")
    ax.set_ylabel("Count")
    ax.tick_params(axis="x", rotation=90)

plt.suptitle('Categorical Feature Distributions')
plt.tight_layout()
plt.show()

You could also look at how categories of specific features correlated with the target. Here are some charts Jonas produced during the hackathon:

In [ ]:
import matplotlib.ticker as mticker

fraction_df = (
    df.groupby('long_term_condition_count_number')['frequent_attender']
    .value_counts(normalize=True)
    .unstack()
)

fraction_df.plot(
	kind='bar',
	stacked=True,
	figsize=(10, 6),
	color=["#0062FF", '#DD8452']
)

plt.legend(title='Frequent Attender')
plt.tight_layout()


ax = plt.gca()
ax.yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))
ax.legend(title='Frequent Attender', labels=['No', 'Yes'])
plt.show()

In [ ]:
fraction_df2 = (
	df.groupby('destination_desc')['frequent_attender']
	.value_counts(normalize=True)
	.unstack()
)

fraction_df2.plot(
	kind='barh',  # horizontal bar plot
	stacked=True,
	figsize=(10, 7),
)

ax = plt.gca()
ax.xaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))
ax.legend(loc='upper left',title='Frequent Attender')
plt.show()

### Looking at the value counts for each feature individually

In [ ]:
df.stated_gender.value_counts()

In [ ]:
df.living_alone.value_counts()

In [ ]:
df.acutely_unwell_flag.value_counts()

In [ ]:
df.all_long_term_condition_count_number.value_counts()

### Getting the list of columns, so that they can be easily copied into the list of columns to drop.

In [ ]:
df.columns

### Dropping the columns that we don't want to keep as features.

In [ ]:
df = df.drop(columns=[
    'nhs_number',
    'organisation_code_provider',
    'organisation_code_commissioner',
    'lsoa_11',
    'index_of_multiple_deprivation_description',
    'accommodation_status_desc',
    'treatment_function_code',
    'acuity_code_approved',
    'gp_practice',
    'gp_practice_code', # high cardinality categorical variable. Take out for now and bring back for CatBoost
    'palliative_care_description',
    'care_home_name',
    'patient_status', # leaks data i.e. if someone has died, can they reattend?
    'all_long_term_conditions',
    'segmentation_bridges_to_health_description',
    'patient_registration_status',
    'all_long_term_condition_count',
    'attendance_category',
    'living_alone'
]
)

In [ ]:
df.info()

In [ ]:
df.palliative_care_flag.value_counts()

Gender was a column that was imported as float values, but it is meant to be categorical. The following cell converts the values to categories, dealing with any unknown values. Converting any NaN values (i.e. Python's equivalent of NULL) into a string results in a 'nan' string, which we need to convert back into an _empty_ string value.

In [ ]:
df['stated_gender'] = df['stated_gender'].astype(str)
df['stated_gender'] = df['stated_gender'].replace('nan','X') # unknown is meant to be X. 9 is unable to classify as male or female.
df['stated_gender'] = df['stated_gender'].str.replace('.0','',regex=False) # regex = False treats the "." literally and not as a regex character.
df['stated_gender'].unique()

### Dropping rows containing missing values

We dropped all rows containing NaN (NULL) values, rather than dealing with the missing values, which requires a lot more thought than we had time for. We were still left with a healthy amount of data to work with.

In [ ]:
df = df.dropna()

Check how many rows and columns we are left with

In [ ]:
df.shape

Identify our target $y$ and feature $X$ columns. The target is the thing you are trying to predict.

In [ ]:
y = df['frequent_attender']
X = df.drop(columns=['frequent_attender'])
X_features = X.columns

### The train-test split

This is where we did our train-test split. Usually this is best done _after_ preprocessing and feature engineering has been completed, so that you end up with the same feature columns in both the train and test sets. It wasn't the end of the world, it just meant a little bit of extra work to get things in order.

Ed got a bit confused since he had recently heard that you need to do the split _before_ any preprocessing / feature engineering, but he missed the nuance that you ought to do the split before any imputation of values that use something like the mean value, because using the whole dataset for this leaks some information from what will become the test set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

### Preprocessing and feature engineering

Converting the 'arrival_datetime' field into the datetime data type.

In [ ]:
X_train['arrival_datetime'] = pd.to_datetime(X_train['arrival_datetime'], dayfirst=True)

In [ ]:
X_test['arrival_datetime'] = pd.to_datetime(X_test['arrival_datetime'], dayfirst=True)

Lucy suggested creating an "arrival outside of core GP hours" flag.

In [ ]:
X_train['arrival_outside_of_core_gp_hours'] = X_train['arrival_datetime'].dt.hour.apply(lambda x: 1 if (x < 8) | (x >= 18) else 0)

In [ ]:
X_test['arrival_outside_of_core_gp_hours'] = X_test['arrival_datetime'].dt.hour.apply(lambda x: 1 if (x < 8) | (x >= 18) else 0)

Create a flag to see whether the original attendance was at the weekend. Could it lead to be treated and assessed differently, with knock-on effects?

In [ ]:
X_test["is_weekend"] = X_test['arrival_datetime'].dt.dayofweek >= 5 # creates a Boolean flag

X_test["is_weekend"] = X_test["is_weekend"].astype(int) # converts the Boolean to a 1 or 0 (keep thing numeric for consistency).

Then we dropped the 'arrival_datetime' field to remove a field that would be predicting the same variation. In hindsight, it might have been better to leave it in for models such as CatBoost, which can handle greater numbers of features & may have been able to find some signal in the arrival datetime.

In [ ]:
X_train = X_train.drop(columns=['arrival_datetime'])

In [ ]:
X_test = X_test.drop(columns=['arrival_datetime'])

We created a "70 years old or older" flag. Paul D identified a correlation between this age band and being a frequent attender. 

In [ ]:
def is_over_70(row):
    if row['age_at_arrival'] > 69:
        return 1
    else:
        return 0

In [ ]:
X_train['over_70'] = X_train.apply(is_over_70, axis = 1)

In [ ]:
X_test['over_70'] = X_test.apply(is_over_70, axis = 1)

James suggested a "four-hour wait" flag

In [ ]:
def four_hour_wait(row):
    if row['departure_time_since_arrival'] > 240:
        return 1
    else:
        return 0
   
X_train['four_hr_Wait'] = X_train.apply(four_hour_wait, axis=1)

In [ ]:
X_test['four_hr_Wait'] = X_test.apply(four_hour_wait, axis=1)

Have a look at which columns we have so far and check they are consistent across train and test sets.

In [ ]:
X_train.columns

In [ ]:
X_test.columns

In [ ]:
X_train.info()

### One-hot encoding categorical features.

We one-hot encoded some categorical features so that they could be treated as numeric fields. This creates a separate column for each category and then records True / False to say whether the value for that row of data falls into that category.

We set a prefix to make the generated columns easily attributable to the original feature. We also set the data type as integer so that it returns a 1 or 0, rather than True / False.

This process generates a separate DataFrame, which needs to be concatenated with the X data (needs to be done with X_train and X_test, if you have already split the data).

In [ ]:
gender_encoded = pd.get_dummies(X_train['stated_gender'], prefix='gender_code', dtype=int)

X_train = pd.concat([X_train, gender_encoded], axis=1)

In [ ]:
gender_encoded_test = pd.get_dummies(X_test['stated_gender'], prefix='gender_code', dtype=int)

X_test = pd.concat([X_test, gender_encoded_test], axis=1)

In [ ]:
X_train

### Correlation of numeric features

At this stage, we could have looked at the correlation between numeric features, to see whether there are any that we could have removed.

Features that correlate highly explain the same variation in the data, and the effect of this can be to _underestimate_ the effect of those features.

We can use `df.corr()` to get a correlation table, or we can use a heatmap visualisation, which will highlight high correlation for us.

In [ ]:
X_train_numeric = X_train.select_dtypes('number')

X_train_numeric.corr()

Thanks to Emile for the following heatmap. The default in `seaborn` returns the entire correlation table colour-coded, but this effectively duplicates the pairs. Emile's version trims off the top-right corner, leaving each feature pair just once.

Ed was going to use a library called `yellowbrick`, which was designed specifically to produce visualisations relating to `scikit-learn` uses. However, it requires a dependency that has recently been deprecated & it would have taken too long to find a workaround.

In [ ]:
# plot covariance rankings
corr = X_train_numeric.corr()  # Compute correlation matrix
mask = np.triu(np.ones_like(corr, dtype=bool), k=1) # Make mask for the upper triangle
f, ax = plt.subplots(figsize=(75, 15)) # Set matplotlib figure 
cmap = sns.diverging_palette(230, 20, as_cmap=True) # Generate a custom diverging colormap


# Draw the heatmap with the mask and correct aspect ratio
ax = sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1, center=0, annot = True,
                 annot_kws={"size": 8}, square=True, linewidths=.5, cbar_kws={"shrink": .5},)

# Add a title
ax.set_title('Correlation Matrix', fontsize=15, fontweight='bold')
plt.show()

You can see that some pairs such as "segmentation_bridges_to_health" and "long_term_condition_number" correlate quite highly. Do we need "long_term_condition_count_number" _and_ "long_term_condition_count_number"? For simpler models, it would probably be best to drop "age_at_arrival" after having created the "over_70" flag.

Sometimes, when you get mutually exclusive categories, it can be worth dropping one of them and renaming the one that you are left with. For example, here you can see that "gender_code_1" (male) and "gender_code_2" (female) are very strongly negatively correlated with each other, as you would expect. You could drop one or the other and have a single "is_male" or "is_female" feature since the negative would imply the other. In actual fact, there are some records for patients with "gender_code_9" (indeterminate), so we probably shouldn't do this with our data since we could lose that information.

# Creating our models

### Dummy Classifier

Acts as a baseline. Predicts the mode target class (in this case "0" for "not a frequent attender") every time.

In [ ]:
from sklearn.dummy import DummyClassifier

dumb_dumb = DummyClassifier()

dumb_dumb.fit(X_train,y_train)

dumb_dumb_probs = dumb_dumb.predict_proba(X_test)[:, 1]


brier_score_loss(y_test, dumb_dumb_probs)


# Logistic regression example

These features were suggested by Paul D as being the most likely to predict whether someone will be a frequent attender, based on his population health expertise.

In [ ]:
log_feats = ['index_of_multiple_deprivation','over_70','segmentation_bridges_to_health']

In [ ]:
log_train = X_train[log_feats] # restrict train and test to just the suggested features.
log_test = X_test[log_feats]

In [ ]:
lgr = LogisticRegression() # instantiate a logistic regression model

lgr.fit(log_train,y_train) # fit the model to the training data

In [ ]:
log_probs = lgr.predict_proba(log_test)[:, 1] # get the prediction probabilities needed to calculate the Brier Score Loss

brier_score_loss(y_test, log_probs) # calculate the Brier Score Loss

We could then have a look at the feature coefficients to see which had the biggest effect on the module.

In [ ]:
coefficients = lgr.coef_[0] # Just returns a numpy array of coefficients without the feature names.

In [ ]:
# this gives us a table that includes the feature names.
# "coeffiecient" gives the value, including whether it is positive or negative.
# "abs_coefficient" gives the absolute value so that we can compare the size of the effect more intuitively.

feature_importance = pd.DataFrame({
    'feature': log_feats,
    'coefficient': coefficients,
    'abs_coefficient': np.abs(coefficients)
}).sort_values(by='abs_coefficient', ascending=False)

In [ ]:
feature_importance

# Random Forest Classifier

We will throw as many numeric features at it as possible.

First, we want to see whether we can group some of the high-cardinality categorical features so that we can one-hot encode some of them.

In [ ]:
string_columns = X_train.select_dtypes(include='string').columns.tolist()

string_columns

In [ ]:
X_train.care_home_status.value_counts() # low cardinality already.

In [ ]:
X_train.ethnic_category.value_counts() # middling cardinality. Records fall overwhelmingly into one category.

We reduced the number of ethnicity categories by grouping them into white, non-white and unknown.

In [ ]:
def ethnicity_grouping(row):
    if row['ethnic_category'] == 'A':
        return 'white'
    elif row['ethnic_category'] == '99':
        return 'unknown'
    else:
        return 'non-white'

X_train['ethnicity_group'] = X_train.apply(ethnicity_grouping, axis=1)
X_test['ethnicity_group'] = X_test.apply(ethnicity_grouping, axis=1)

We dropped the original feature.

In [ ]:
X_train = X_train.drop(columns=['ethnic_category'])
X_test = X_test.drop(columns=['ethnic_category'])

Most of the records fell into two arrival mode categories, so we groupen them by those two plus "other".

In [ ]:
X_train.arrival_mode_desc.value_counts()

In [ ]:
def arrival_mode_grouping(row):
    if row['arrival_mode_desc'] == 'Arrival by own transport (finding)':
        return 'own_transport'
    elif row['arrival_mode_desc'] == 'Arrival by emergency road ambulance (finding)':
        return 'ambulance'
    else:
        return 'other'
    
X_train['arrival_mode_grouping'] = X_train.apply(arrival_mode_grouping, axis=1)
X_test['arrival_mode_grouping'] = X_test.apply(arrival_mode_grouping, axis=1)

In [ ]:
X_train = X_train.drop(columns=['arrival_mode_desc'])
X_test = X_test.drop(columns=['arrival_mode_desc'])

Again, discharge status was mainly in one category, so we created a "completed_treatment" numeric flag.

In [ ]:
X_train.discharge_status_desc.value_counts()

In [ ]:
def completed_treatment(row):
    if row['discharge_status_desc'] == 'Treatment completed (situation)':
        return 1
    else:
        return 0
    
X_train['completed_treatment'] = X_train.apply(completed_treatment, axis=1)
X_test['completed_treatment'] = X_test.apply(completed_treatment, axis=1)

In [ ]:
X_train = X_train.drop(columns=['discharge_status_desc'])
X_test = X_test.drop(columns=['discharge_status_desc'])

Because we were running out of time on the day, some of the categorical features were left as they were. They will be used when we come to the CatBoost model, which is capable of dealing with high-cardinality categorical features.

In [ ]:
X_train.destination_desc.value_counts()

In [ ]:
X_train.acuity_desc.value_counts()

Now we will one-hot encode the new categorical groupings.

In [ ]:
ethnicity_encoded = pd.get_dummies(X_train['ethnicity_group'], prefix='ethnicity_group', dtype=int)

X_train = pd.concat([X_train, ethnicity_encoded], axis=1)

In [ ]:
ethnicity_encoded_test = pd.get_dummies(X_test['ethnicity_group'], prefix='ethnicity_group', dtype=int)

X_test = pd.concat([X_test, ethnicity_encoded_test], axis=1)

In [ ]:
X_train = X_train.drop(columns=['ethnicity_group'])
X_test = X_test.drop(columns=['ethnicity_group'])

In [ ]:
arrival_mode_encoded = pd.get_dummies(X_train['arrival_mode_grouping'], prefix='arrival_mode_grouping', dtype=int)

X_train = pd.concat([X_train, arrival_mode_encoded], axis=1)

In [ ]:
arrival_mode_encoded_test = pd.get_dummies(X_test['arrival_mode_grouping'], prefix='arrival_mode_grouping', dtype=int)

X_test = pd.concat([X_test, arrival_mode_encoded_test], axis=1)

In [ ]:
X_train = X_train.drop(columns=['arrival_mode_grouping'])
X_test = X_test.drop(columns=['arrival_mode_grouping'])

In [ ]:
completed_treatment_encoded = pd.get_dummies(X_train['completed_treatment'], prefix='completed_treatment', dtype=int)

X_train = pd.concat([X_train, completed_treatment_encoded], axis=1)

In [ ]:
completed_treatment_encoded_test = pd.get_dummies(X_test['completed_treatment'], prefix='completed_treatment', dtype=int)

X_test = pd.concat([X_test, completed_treatment_encoded_test], axis=1)

In [ ]:
X_train = X_train.drop(columns=['completed_treatment'])
X_test = X_test.drop(columns=['completed_treatment'])

Now we will start building the Random Forest Classifier model, using only the numeric features.

In [ ]:
rf_feats = X_train.select_dtypes('number').columns

rf_feats

In [ ]:
rf_train = X_train[rf_feats]
rf_test = X_test[rf_feats]

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(rf_train, y_train)
rf_probs = rf.predict_proba(rf_test)[:, 1]
brier_score_loss(y_test, rf_probs)

This appears to be worse than the dummy classifier. This could be because it is struggling with the imbalanced target class, where there are far more patients who are not frequent attenders than are. The dummy classifier potentially did better simply because it predicted "0" all the time.

This is where we might want to score each model for accuracy, precision and recall.

#### Accuracy

Proportion of correct predictions from all predictions:

$\text{Accuracy} = \frac{TP + TN}{TP + TN + FP + FN}$

Not sufficient on its own, because if you have 1 positive in 10,000, you could get very high accuracy by predicting negative all the time.

#### Recall

Also known as _sensitivity_. The percentage of positive values correctly classified.

$\text{Recall} = \frac{TP}{TP + FN}$

_How many relevant results are returned?_

#### Precision

Percentage of positive predictions that were correct.

$\text{Precision} = \frac{TP}{TP + FP}$ 

_How relevant are the results?_

#### F1

"Combined" precision and recall. It's the _harmonic mean_ of the two

$\text{F1 Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}$

or

$\text{F1 Score} = 2 \times \frac{TP}{2TP + FP + FN}$

### Feature Importances. 

They indicate that treatment distance was considered the most important, which is surprising. Age and IMD still come out quite high. The model probably would have performed better if we had dropped more of the correlating features.

In [ ]:
for col, val in sorted(
    zip(
        rf_feats, 
        rf.feature_importances_
        ), 
    key=lambda x: x[1], 
    reverse=True):
        print(f'{col:20}{val:10.3f}')

# Catboost

This will use all of the remaining features in X_train (rather than just the numeric features). We will also drop "age_at_arrival" and just keep the "over_70" flag.

We will just use the default settings for now.

In [ ]:
X_train = X_train.drop(columns=['age_at_arrival','stated_gender']) # stated_gender can go, too, because we have one-hot encoded it
X_test = X_test.drop(columns=['age_at_arrival','stated_gender'])

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
categorical_features = ['destination_desc', 'acuity_desc', 'care_home_status']

In [ ]:
cat = CatBoostClassifier(cat_features=categorical_features)
cat.fit(X_train, y_train)
cat_probs = cat.predict_proba(X_test)[:, 1]
brier_score_loss(y_test, cat_probs)

Quickly try hyperparameter tuning with Optuna. The number of iterations has been fixed, and the number of trials (`n_trials`) and `early_stopping_rounds` have been set quite low, otherwise it takes quite a long time to run on this large dataset.

In [ ]:
import optuna

In [ ]:
def objective(trial):
    param = {
        'iterations': 100,
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-3, 10),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
        'leaf_estimation_method': trial.suggest_categorical('leaf_estimation_method', ['Newton', 'Gradient']),
    }
    
    catTuna = CatBoostClassifier(**param, verbose=0, cat_features=categorical_features)
    catTuna.fit(X_train, y_train, early_stopping_rounds=20) # stop after 20 rounds if the score stops improving.
    probTuna = cat.predict_proba(X_test)[:, 1]
    brierCat = brier_score_loss(y_test, probTuna)
    return brierCat

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

# Get best parameters
best_paramsTuna = study.best_params

# Train final model with best parameters
best_cat = CatBoostClassifier(**best_paramsTuna, cat_features=categorical_features, verbose=0)
best_cat.fit(X_train, y_train)
final_probTuna = best_cat.predict_proba(X_test)[:, 1]
finalBrierCat = brier_score_loss(y_test, final_probTuna)

It didn't do any better than the default parameters, probably because we only let it run a few iterations.